In [ ]:
#!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#!pip install transformers requests beautifulsoup4 pandas numpy

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

D:\workspace\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [3]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

4

In [10]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [8]:
reviews = [
    "The food was amazing, and the service was excellent!",
    "I had a terrible experience. The food was cold and the staff was rude.",
    "Great ambiance and friendly staff. The coffee was delicious.",
    "The restaurant was clean and well-maintained. I highly recommend it.",
    "Disappointing experience. The portion sizes were small, and the prices were high.",
    "Fantastic place! The atmosphere is cozy, and the menu has a good variety of options.",
    "I didn't enjoy my visit. The food took too long to arrive, and it wasn't worth the wait.",
    "Lovely spot for brunch. The pancakes were fluffy, and the coffee was brewed to perfection.",
    "The service was slow, but the food made up for it. Overall, a decent experience.",
    "Not impressed. The food lacked flavor, and the prices were unjustified."
]


In [11]:
for review in reviews:
    # Tokenize the review text
    tokens = tokenizer.encode(review, return_tensors='pt')
    
    # Pass the tokens through the model
    result = model(tokens)
    
    # Get the predicted sentiment (1 for negative, 5 for positive)
    sentiment = int(torch.argmax(result.logits)) + 1
    
    # Print the review and its sentiment
    print(f"Review: {review}")
    print(f"Sentiment: {sentiment}\n")

Review: The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.
Sentiment: 4

Review: Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!
Sentiment: 5

Review: Ordered coffee which Combines cozy ambiance with delicious co

In [12]:
def predict_sentiment(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    sentiment_label = int(torch.argmax(result.logits)) + 1
    return sentiment_label

In [13]:
sentiment_names = {
    1: "Very Negative",
    2: "Negative",
    3: "Neutral",
    4: "Positive",
    5: "Very Positive"
}

In [15]:
sentiments = [predict_sentiment(review) for review in sample_reviews]

In [16]:
sample_reviews = [
    "The food was terrible, would not recommend.",
    "Service was slow and the staff was rude.",
    "Average experience, nothing special.",
    "Great ambiance and delicious food!",
    "Best cafe in town, highly recommended!"
]
for i, review in enumerate(sample_reviews):
    print(f"Review {i+1}: {review} - Sentiment: {sentiment_names[sentiments[i]]}")

Review 1: The food was terrible, would not recommend. - Sentiment: Very Negative
Review 2: Service was slow and the staff was rude. - Sentiment: Negative
Review 3: Average experience, nothing special. - Sentiment: Neutral
Review 4: Great ambiance and delicious food! - Sentiment: Very Positive
Review 5: Best cafe in town, highly recommended! - Sentiment: Very Positive


In [24]:
# Define the directory where you want to save the model
output_dir = r"D:\workspace\codespace\Sentimental analysis\bert_sentiment_model"

# Save the tokenizer and model to the specified directory
tokenizer.save_pretrained(output_dir)
model.save_pretrained(output_dir)

print("Model and tokenizer saved successfully to:", output_dir)


Model and tokenizer saved successfully to: D:\workspace\codespace\Sentimental analysis\bert_sentiment_model


In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define the directory where the model and tokenizer are saved
saved_model_dir = r"D:\workspace\codespace\Sentimental analysis\bert_sentiment_model"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(saved_model_dir)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(saved_model_dir)

# List of reviews
reviews = ["This restaurant is amazing!",
           "The food was delicious but the service was slow.",
           "I had a terrible experience at this place."]

# Tokenize the reviews
encoded_inputs = tokenizer(reviews, padding=True, truncation=True, return_tensors='pt')

# Perform inference
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Extract predicted sentiment scores
predicted_scores = torch.softmax(outputs.logits, dim=1)

# Convert scores to human-readable labels
sentiment_labels = ["Very Negative","Negative","Neutral","Positive","Very Positive"]
predicted_labels = [sentiment_labels[p.argmax().item()] for p in predicted_scores]

# Print predicted sentiment labels for each review
for review, label in zip(reviews, predicted_labels):
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {label}")
    print()


Review: This restaurant is amazing!
Predicted Sentiment: Very Positive

Review: The food was delicious but the service was slow.
Predicted Sentiment: Neutral

Review: I had a terrible experience at this place.
Predicted Sentiment: Very Negative

